In [1]:
from TASD.translate_sequence_to_tuples import get_tuples_from_prediction
import numpy as np
import constants


predictions = np.load("predictions_bad.npy")
labels = np.load("labels_bad.npy")


In [4]:
from transformers import AutoTokenizer
from datasets import load_metric
import numpy as np
import constants
import nltk

# Source: https://medium.com/nlplanet/a-full-guide-to-finetuning-t5-for-text2text-and-building-a-demo-with-streamlit-c72009631887
def compute_metrics(predictions, labels):
    # Evtl später übertragen
    tokenizer = AutoTokenizer.from_pretrained(constants.MODEL_NAME_TASD)
    metric = load_metric("rouge")

    predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)

    decoded_preds = tokenizer.batch_decode(
        predictions, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip()))
                     for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip()))
                      for label in decoded_labels]
    
    # Extract Tuples
    pred_tuples = [get_tuples_from_prediction(example) for example in decoded_preds]
    labels_tuples = [get_tuples_from_prediction(example) for example in decoded_labels]

    print(len(pred_tuples), len(labels_tuples))

    
    

    # Text Based Metrics 
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [5]:
compute_metrics(predictions, labels)

/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow_m1/lib/python3.11/site-packages/transformers/models/t5/tokenization_t5_fast.py:158: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


500 500


{'rouge1': 69.9538,
 'rouge2': 64.8427,
 'rougeL': 69.5201,
 'rougeLsum': 69.629,
 'gen_len': 24.136}